## Question 1  
NO, if the weights of all variable is the same. Backpropagation will yield the same results and all of the neuron will have the same weights. This lead to the symmetry of the model and it will behave like one neuron model.

## Question 2  
Yes, bias terms can be initialized as zero.

## Question 3. 
1. SELU can perform self normalization of the input and output. This behavior can mitigate vanishing gradient and exploding gradient.  
2. SELU can be differentiate if z < 0. This, also, can mitigate vanishing gradient.  
3. SELU can also differentiate at z = 0. This property make SELU more stable in converging to global optimal.  
4. It always has a non-zero derivative, which avoids dying units.

## Question 4. 
1. SELU can do self normalization if the model is dense, it also can be diff if z<=0. Thus it is preferable on general case.  
2. Leaky ReLU is the improve ment on ReLU in the way that it can be diff if z < 0. Thus it can mitigate vanishing gradient. However, this introduced one more parameter to be tweaked which is alpha.  
3. ReLU in practice give the model good performance and it very fast to compute. Thus, if you want to speed things up with reasonable performance, ReLU is a good choice.  
4. Tanh and logistic shouldn't be used in hidden layer, since it is saturated acitvation func. It will introduce vanishing gradient. But it can be used at the output layer for classification task.  
5. Softmax will be used at output layer for multilabel classification tasks.

## Question 5  
If we set the momentum too close to 1. The algorithm will quickly pick up a lot of speed (assume it is negative for now), then it shoot right pass the global optimum. After passing the global optimum, the speed will slowdown (because the current gradient is positive at cancel out the accumulate negative gradient) and comeback. It will osscilate this way before converging. 

## Question 6. 
* To produce sparse model, we have to implement l1 normalization which eliminate the unnecessary weights.  
* The other way is to implement tf Model Optimization Toolkit.

## Question 7. 
* Dropout slows down training phase, because it randomly eliminate the neuron in each steps and make the model harder to fit to the given data. However, at testing phase, the dropout will be cancel out. So it is not slowdown testing at all.  
* MC dropout slows down train phase as well as typical dropout. It also slows down testing phase, since the concept of MC is to testing multiple time while dropout is still on. The final results will be averaged from multiple testing.

## Question 8. 